In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
import torchvision
import torchvision.transforms as transforms
import cv2
from torch.utils.data import Dataset, DataLoader

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import Adam

from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import glob
from tqdm.notebook import tqdm, trange

mask_paths = glob.glob('kaggle_3m/*/*_mask.tif')
empty_masks = []
non_empty_masks = []

for mask_path in tqdm(mask_paths):
    mask = np.array(Image.open(mask_path))
    if (mask == 0).all():
        empty_masks.append(mask_path)
    else:
        non_empty_masks.append(mask_path)

        
masks_paths = non_empty_masks + [empty_masks[idx] for idx in np.random.choice(len(empty_masks), 200)]
image_paths = [mask_path.replace('_mask', '') for mask_path in mask_paths]

path_df = pd.DataFrame({'image':image_paths, 'mask':mask_paths})


  0%|          | 0/3929 [00:00<?, ?it/s]

In [2]:
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
NUM_CHANNELS = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
class MRIDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
        self.preprocess_image = transforms.Compose([
            transforms.PILToTensor(),
            transforms.Resize((IMAGE_WIDTH, IMAGE_HEIGHT)),
            transforms.ConvertImageDtype(torch.float),
        ])
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if idx >= self.__len__():
            raise KeyError
            
        image_path = self.df.loc[idx, 'image']
        image = Image.open(image_path)
        image = self.preprocess_image(image)
        
        mask_path = self.df.loc[idx, 'mask']
        mask = Image.open(mask_path)
        mask = self.preprocess_image(mask)
        
        return image, mask
    
train, test_val = train_test_split(path_df, test_size = 0.3)
test, val = train_test_split(test_val, test_size= 0.25)

train = MRIDataset(train)
test = MRIDataset(test)
val = MRIDataset(val)

train = DataLoader(train, batch_size=16)
test = DataLoader(test, batch_size=16)
val = DataLoader(val, batch_size=16)

In [4]:

import torch
import torch.nn as nn
from torch.optim import Adam

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from tqdm.notebook import tqdm, trange

IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
NUM_CHANNELS = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
class ResConvBlock(nn.Module):
    def __init__(self, input_features, output_features, dropout, kernel_size=3 ):
        super().__init__()
        self.conv1 = nn.Conv2d(input_features, output_features, kernel_size, padding='same')
        self.conv2 = nn.Conv2d(output_features, output_features, kernel_size, padding='same')
        self.batch_norm = nn.BatchNorm2d(output_features)
        
        self.res_conv = nn.Conv2d(input_features, output_features, 1, padding='same')
        self.res_batch_norm = nn.BatchNorm2d(output_features)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.con_batch_norm = nn.BatchNorm2d(output_features)
        
    def forward(self, x):
        res_x = x
        
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.batch_norm(x)
        
        res_x = self.res_conv(res_x)
        x = self.relu(x)
        res_x = self.res_batch_norm(res_x)
        
        x = torch.add(x, res_x)
        x = self.con_batch_norm(x)
        
        del res_x
        
        return x
   

def test_model():
    x = torch.rand(16, 3, 256, 256).to(device)
    model = ResConvBlock(3, 16, 0.1).to(device)
    y = model(x)
    
    return y.shape, torch.isnan(y).sum()

test_model()

(torch.Size([16, 16, 256, 256]), tensor(0))

In [6]:
class Encoder(nn.Module):
    def __init__(self, input_features, output_features, dropout, kernel_size=3):
        super().__init__()
        self.conv_block = ResConvBlock(input_features, output_features, dropout, kernel_size)
        self.downsampler = nn.MaxPool2d((2, 2))
    
    def forward(self, x):
        c = self.conv_block(x)
        x = self.downsampler(c)
        
        return x, c
    
def test_model():
    x = torch.rand(16, 3, 256, 256).to(device)
    model = Encoder(3, 16, 0.1).to(device)
    c, y = model(x)
    
    return y.shape, torch.isnan(y).sum(), c.shape, torch.isnan(c).sum()

test_model()    


(torch.Size([16, 16, 256, 256]),
 tensor(0),
 torch.Size([16, 16, 128, 128]),
 tensor(0))

In [7]:
class Decoder(nn.Module):
    def __init__(self, input_features, output_features, dropout, kernel_size=3):
        super().__init__()
        self.upsampler = nn.ConvTranspose2d(input_features, input_features, 2, stride=2)
        self.conv_block = ResConvBlock(2*input_features, output_features, dropout, kernel_size)
   
    def forward(self, x, c):
        x = self.upsampler(x)
        x = torch.concat([x, c], dim=1)
        x = self.conv_block(x)
        
        return x
    
def test_model():
    x = torch.rand(1, 16, 128, 128).to(device)
    c = torch.rand(1, 16, 256, 256).to(device)
    model = Decoder(16, 3, 0.1).to(device)
    y = model(x, c)
    
    return y.shape, torch.isnan(y).sum()

test_model()

(torch.Size([1, 3, 256, 256]), tensor(0))

In [8]:
%%time
class ResUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoders = nn.ModuleList([Encoder(1, 16, 0.1)]+[Encoder(16*(2**i), 16*(2**(i+1)), 0.1*i) for i in range(3)])
        
        self.pointwise1 = nn.Conv2d(128, 256, 1)
        self.relu = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm2d(256)
        
        self.pointwise2 = nn.Conv2d(256, 128, 1)
        self.relu = nn.ReLU()
        self.batch_norm2 = nn.BatchNorm2d(128)
        
        self.decoders = nn.ModuleList([Decoder(16*(2**(3-i)), 16*(2**(2-i)), 0.1*(3-i)) for i in range(3)]+[Decoder(16, 1, 0.1)])
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        conv_outputs = []
        
        for layer in self.encoders:
            x , c = layer(x)
            conv_outputs.append(c)
                    
        x = self.pointwise1(x)
        x = self.relu(x)
        x = self.batch_norm1(x)
        
        x = self.pointwise2(x)
        x = self.relu(x)
        x = self.batch_norm2(x)
        
        for i, layer in enumerate(self.decoders):
            c = conv_outputs[3 - i]
            x = layer(x, c)
        
        del conv_outputs
        x = self.sigmoid(x)
        
        return x
    
def test_model():
    model = ResUNet().to(device)
    x = torch.rand(1, 1, 256, 256).to(device)
    y = model(x)
    
    return y.shape, torch.sum(torch.isnan(y))

test_model()

CPU times: total: 188 ms
Wall time: 182 ms


(torch.Size([1, 1, 256, 256]), tensor(0))

In [9]:
model = ResUNet().to(device)
lr = 1e-3
optimizer = Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

In [10]:
images, masks = next(iter(test))
preds = model(images.to(device))

plt.subplots(10, 2, figsize=(10, 50))
for i in range(10):
    plt.subplot(10, 2, (i+1)*2 - 1)
    plt.imshow(preds[i, 0,:, :].detach().cpu().numpy())
    plt.subplot(10, 2, (i+1)*2)
    plt.imshow(masks[i, 0,:, :].detach().cpu().numpy())
plt.show()    

RuntimeError: Given groups=1, weight of size [16, 1, 3, 3], expected input[16, 3, 256, 256] to have 1 channels, but got 3 channels instead

In [ ]:
EPOCHS = 80
min_val_loss = np.inf


MODEL_CHK_PATH = r'C:\\Users\\roee hilel\\Computer\Documents\\vs-code\\apllied_statistics\\Brain-MRI\\check_points\\'
os.makedirs(MODEL_CHK_PATH, exist_ok=True)

for epoch in trange(EPOCHS):
    for i, (image, mask) in enumerate(tqdm(train)):
        image, mask = image.to(device), mask.to(device)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, mask)
        loss.backward()
        optimizer.step()
        

    for j, (image, mask) in enumerate(tqdm(val)):
        image, mask = image.to(device), mask.to(device)
        avg_loss = []
                
        with torch.no_grad():
            output = model(image)
            loss = criterion(output, mask)
            avg_loss.append(loss.cpu().numpy())
    avg_loss = np.mean(avg_loss)
    
    if avg_loss < min_val_loss:
        torch.save(model.state_dict(), MODEL_CHK_PATH + 'model')
        print(f'val_loss decreased by {min_val_loss - avg_loss}')
        min_val_loss = avg_loss
            
    print(f'EPOCH: {epoch}, VAL_LOSS: {avg_loss}')
                    


  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/172 [00:00<?, ?it/s]

RuntimeError: Given groups=1, weight of size [16, 1, 3, 3], expected input[16, 3, 256, 256] to have 1 channels, but got 3 channels instead

In [ ]:
import numpy as np
import os

EPOCHS = 80
min_val_loss = np.inf

MODEL_CHK_PATH = r'C:\\Users\\roee hilel\\Computer\Documents\\vs-code\\apllied_statistics\\Brain-MRI\\check_points\\'
os.makedirs(MODEL_CHK_PATH, exist_ok=True)


for epoch in range(EPOCHS):
    # Training phase
    for i, (image, mask, label) in enumerate(train_dl):
        image, mask, label = image.to(device), mask.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, mask)
        loss.backward()
        optimizer.step()

    # Validation phase
    avg_loss = []
    for j, (image, mask) in enumerate(val_dl):
        image, mask = image.to(device), mask.to(device)
                
        with torch.no_grad():
            output = model(image)
            loss = criterion(output, mask)
            avg_loss.append(loss.cpu().numpy())
    
    avg_loss = np.mean(avg_loss)
    
    # Checkpointing
    if avg_loss < min_val_loss:
        torch.save(model.state_dict(), os.path.join(MODEL_CHK_PATH, 'model.pth'))
        print(f'Validation loss decreased by {min_val_loss - avg_loss}')
        min_val_loss = avg_loss
            
    print(f'EPOCH: {epoch + 1}/{EPOCHS}, Validation Loss: {avg_loss}')


RuntimeError: Given groups=1, weight of size [16, 1, 3, 3], expected input[32, 3, 256, 256] to have 1 channels, but got 3 channels instead

In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

dataloader = torch.utils.data.DataLoader(train_ds, batch_size=4, shuffle=True)

for epoch in range(7):
    correct = 0
    total = 0
    current_predicted = 0
    current_total = 0
    for i, data in enumerate(dataloader, 0):

        inputs, _,labels = data
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)

        optimizer.zero_grad()

        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        predicted = torch.round(outputs.data) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        current_total += labels.size(0)
        current_predicted += (predicted == labels).sum().item()
        if i % 100 == 0:
            print('epoch',epoch)
            print('Current accaracy: %d %%' % (100 * current_predicted / current_total))
            print('Accuracy of the network on the train images: %d %%' % (100 * correct / total))
            current_predicted = 0
            current_total = 0


print('Finished Training')


RuntimeError: Given groups=1, weight of size [16, 1, 3, 3], expected input[4, 3, 256, 256] to have 1 channels, but got 3 channels instead